# Time Series Classification, Regression, Clustering & More

### Overview of this notebook

* Introduction to time series classification, regression, clustering
* `sktime` data format fo "time series panels" = collections of time series
* Basic vignettes for TSC, TSR, TSCl
* Advanced vignettes - pipelines, ensembles, tuning


Deal with *collections of time series* = "panel data"

Classification = try to assign one *category* per time series, after training on time series/category examples

- Example: Daily energy consumption profile over time - Predict season, e.g., winter/summer, or type of consumer

Regression = try to assign one continuous *numerical value* per time series, after training on time series/category examples

- Example: Temperature/pressure/time profile of chemical reactor - Predict total purity (fraction of 1)

Clustering = put different time series in a small number of similarity buckets

- Example: Service Level Agreement (SLA) Breaches - Group the collected Panel data to identify common reasons for SLA failures

Time Series Classification:

<img src="./img/tsc.png" width="600" alt="time series classification"> [<i>&#x200B;</i>](./img/tsc.png)

In [ ]:
import numpy as np
import pandas as pd

# Increase display width
pd.set_option("display.width", 1000)

## 2.1 Panel data - `sktime` data formats <a name="panel"></a>

`Panel` is an abstract data type where the values are observed for:

* `instance`, e.g., patient
* `variable`, e.g., blood pressure, body temperature of the patient
* `time`/`index`, e.g., January 12, 2023 (usually but not necessarily a time index!)

One value X is: "patient 'A' had blood pressure 'X' on January 12, 2023"

Time series classification, regression, clustering: slices `Panel` data by instance


Preferred format 1: `pd.DataFrame` with 2-level `MultiIndex`, (instance, time) and columns: variables

Preferred format 2: 3D `np.ndarray` with index (instance, variable, time)

* `sktime` supports and recognizes multiple data formats for convenience and internal use, e.g., `dask`, `xarray`
* abstract data type = "scitype"; in-memory specification = "mtype"
* More information in tutorial on [in-memory data representations and data loading](https://www.sktime.net/en/latest/examples/AA_datatypes_and_datasets.html#In-memory-data-representations-and-data-loading)

### 2.1.1 Preferred format 1 - `pd-multiindex` specification

`pd-multiindex` = `pd.DataFrame` with 2-level `MultiIndex`, (instance, time) and columns: variables

In [ ]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in pd-multiindex mtype
X, _ = load_italy_power_demand(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["total_power_demand"]
X.index.names = ["day_ID", "hour_of_day"]

The Italy power demand dataset has:

* 1096 individual time series instances = single days of total power demand (mean subtracted)
* one single variable per time series instances, `total_power_demand`
    * total power demand on that day, in that hourly period
    * Since there's only one column, it is a univariate dataset
* individual time series are observed at 24 time (period) points (the same number for all instances)

In the dataset, days are jumbled and of different scope (independent sampling).
* considered independent - because `hour_of_day` in one sample doesn't affect `hour_of_day` in another
* for task, e.g., "identify season or weekday/weekend from pattern"

In [ ]:
X

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in pd-multiindex mtype
X, _ = load_basic_motions(return_type="pd-multiindex")

# renaming columns for illustrative purposes
X.columns = ["accel_1", "accel_2", "accel_3", "gyro_1", "gyro_2", "gyro_3"]
X.index.names = ["trial_no", "timepoint"]

The basic motions dataset has:

* 80 individual time series instances = trials = person engaging in an activity like running, badminton, etc.
* six variables per time series instance, `dim_0` to `dim_5` (renamed according to the values they represent)
    * 3 accelerometer and 3 gyrometer measurements
    * hence a multivariate dataset
* individual time series are observed at 100 time points (the same number for all instances)

In [ ]:
# The outermost index represents the instance number
# whereas the inner index represents the index of the particular index
# within that instance.
X

pandas provides a simple way to access a range of value in the multi-indexed dataframe:

In [ ]:
# Select:
# * the fourth variable (gyroscope 1)
# * of the first instance (trial 1 = 0 in python)
# * values at all 100 timestamps
#
X.loc[0, "gyro_1"]

Or if you want to access the individual values:

In [ ]:
# Select:
# * the fifth time time point (5 = 4 in python, because of 0-indexing)
# * the third variable (accelerometer 3)
# * of the forty-third instance (trial 43 = 42 in python)

X.loc[(42, 4), "accel_3"]

### 2.1.2 preferred format 2 - `numpy3D` specification

`numpy3D` = 3D `np.ndarray` with index (instance, variable, time)

instance/time index is interpreted as integer

IMPORTANT: unlike `pd-multiindex`, this assumes:

* all individual series have the same length
* all individual series have the same index

In [ ]:
from sktime.datasets import load_italy_power_demand

# load an example time series panel in numpy mtype
X, _ = load_italy_power_demand(return_type="numpy3D")

The Italy power demand dataset has:

* 1096 individual time series instances = single days of total power demand (mean subtracted)
* one single variable per time series instances, unnamed in numpy
* individual time series are observed at 24 time (period) points (the same number for all instances)

In [ ]:
# (num_instances, num_variables, length)
X.shape

In [ ]:
from sktime.datasets import load_basic_motions

# load an example time series panel in numpy mtype
X, _ = load_basic_motions(return_type="numpy3D")

The basic motions dataset has:

* 80 individual time series instances = trials = person engaging in activity (running, badminton, etc)
* six variables per time series instance, unnamed in numpy
* individual time series are observed at 100 time points (the same number for all instances)

In [ ]:
X.shape

## 2.2 Time Series Classification, Regression, Clustering - Basic Vignettes

Above tasks are very similar to "tabular" classification, regression, clustering, as in `sklearn`

Main distinction:
* in "tabular" classification etc, one (feature) instance row vector of features
* in TSC, one (feature) instance is a full time series, possibly unequal length, distinct index set

![](./img/tsc.png)


More formally:

* "tabular" classification:
    * training pairs $(x_1, y_1), \dots, (x_n, y_n)$
        * where $x_i$ are rows of a `pd.DataFrame` (same col types)
        * and $y_i \in \mathcal{C}$ for a finite set $\mathcal{C}$
    * is used to train a classifier that
        * for a new `pd.DataFrame` row $x_*$
        * predicts $y_* \in \mathcal{C}$


* time series classification:
    * training pairs $(x_1, y_1), \dots, (x_n, y_n)$
        * where $x_i$ are time series instances, from a certain domain
        * and $y_i \in \mathcal{C}$ for a finite set $\mathcal{C}$
    * is used to train a classifier that
        * for a new time series instance $x_*$
        * predicts $y_* \in \mathcal{C}$

very similar for time series regression, clustering - exercise left to reader :-)

`sktime` design implications:

* need representation of collections of time series (panels),
    see tutorial [In-memory data representations and data loading](AA_datatypes_and_datasets.ipynb) for more details on representation of Panel data.
    * same as in "adjacent" learning tasks, e.g., panel forecasting
    * same as for transformation estimators
* algorithms that use sequentiality, can deal with unequal length, missing values etc 
* algorithms usually based on distances or kernels between time series - need to cover that in framework
* but we can use familiar `fit` / `predict` and `scikit-learn` / `scikit-base` interface!

### 2.2.3 Time Series Classification - deployment vignette

Basic deployment vignette for TSC:

1. load/setup training data, `X` in a `Panel` (more specifically `numpy3D`) format, `y` as 1D `np.ndarray`
2. load/setup new data for prediction (can be done after 3 too)
3. specify the classifier using `sklearn`-like syntax
4. fit classifier to training data, `fit(X, y)`
5. predict labels on new data, `predict(X_new)`

In [ ]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_type="numpy3D")
X_new, _ = load_italy_power_demand(split="test", return_type="numpy3D")

In [ ]:
# this is in numpy3D format, but could also be pd-multiindex or other
X_train.shape

In [ ]:
# y is a 1D np.ndarray of labels - same length as number of instances in X_train
y_train.shape

In [ ]:
# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

# example 1 - 3-NN with simple dynamic time warping distance (requires numba)
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3)

# example 2 - custom distance:
# 3-nearest neighbour classifier with Euclidean distance (on flattened time series)
# (requires scipy)
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=eucl_dist)

we could specify any `sktime` classifier here - the rest remains the same!

In [ ]:
# all classifiers is scikit-learn / scikit-base compatible!
# nested parameter interface via get_params, set_params
clf.get_params()

In [ ]:
# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

In [ ]:
# the classifier is now fitted
clf.is_fitted

In [ ]:
# and we can inspect fitted parameters if we like
clf.get_fitted_params()

In [ ]:
# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [ ]:
# y_pred is an 1D np.ndarray, similar to sklearn classification output
y_pred

In [ ]:
# predictions and unique counts, for illustration
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

all together in one cell:

In [ ]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_type="numpy3D")
X_new, _ = load_italy_power_demand(split="test", return_type="numpy3D")

# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=eucl_dist)

# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

### 2.2.4 Time Series Classification - simple evaluation vignette

Evaluation is similar to `sklearn` classifiers - we split a dataset and evaluate performance on the test set.

This includes as additional steps:

* splitting the initial, historical data, e.g., using `train_test_split`
* comparing predictions with a held out data set

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_italy_power_demand

# data should be split into train/test
X_train, y_train = load_italy_power_demand(split="train", return_type="numpy3D")
X_test, y_test = load_italy_power_demand(split="test", return_type="numpy3D")

# step 3-5 are the same

from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=eucl_dist)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# for simplest evaluation, compare ground truth to predictions
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

### 2.2.5 Multivariate Time Series Classification - basic vignettes

In time series classification, handling multivariate data can be crucial when each variable or feature impacts classification. This section introduces three methods to deal with multivariate data.

1. Classifiers with Native Multivariate Data support: Some estimators natively support multivaridate data. An example 
of this is `TimeSeriesForestClassifier`. This capability is also indicated by `capability: multivariate` tag. Refer [Section 2.3](#23-searching-for-estimators-estimator-tags) for more.
2. `IndepDist`: Aggregates univariate distances across multiple variables to calculate an overall distance for multivariate data. Mathematically, let $d(x_i​,y_i​)$ be a univariate distance function applied to a variable i. The aggregated multivariate distance $d_y(x,y)$ can be defined as $$d_g​(x,y)=g(d(x_1​,y_1​),d(x_2​,y_2​),…,d(x_D​,y_D​))$$ where g is an aggregation function (eg. sum, mean). This approach is beneficial when each variable provides unique information for classification. Refer this [link](https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.dists_kernels.indep.IndepDist.html) for a detailed walkthrough.

3. `CombinedDistance`: This class enables combining multiple univariate distances or kernel transformers through an arithmetic operation (e.g addition, multiplication) on their respective distance matrices. Suppose we have N base distance matrices $dist_1$, $dist_2$, $dist_3$,..., $dist_4$ calculated from different distance transformers applied to a dataset. For an operation $f$ (eg. `np.sum` or `np.mean`) the `CombinedDistance` outputs a single matrix $dist$ where each entry is calculated as $$dist[i,j] = f(dist_1​[i,j],dist_2​[i,j],…,dist_N​[i,j])$$ This formula provides flexibility to test out various transformations. Refer this [link](https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.dists_kernels.algebra.CombinedDistance.html) for a detailed walkthrough.

For further details on distance methods, refer to [Tutorial](https://www.sktime.net/en/latest/examples/06_distances_kernels_alignment.html)

`TimeSeriesForestClassifier` is a good example of an estimator which handles multivariate data natively.

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head

# step 1-- prepare a dataset (multivariate for demonstration)
X_train, y_train = load_arrow_head(split="train")
X_new, _ = load_arrow_head(split="test")

# step 2-- define the TimeSeriesForestClassifier
tsf = TimeSeriesForestClassifier()

# step 3-- train the classifier on the data
tsf.fit(X_train, y_train)

# step 4-- predict labels on the new data
y_pred = tsf.predict(X_new[:5])

In [ ]:
y_pred

Example of `IndepDist` with `sum` Aggregation Function on a Multivariate dataset.

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head
from sktime.dists_kernels.compose_tab_to_panel import FlatDist
from sktime.dists_kernels.indep import IndepDist
from sktime.dists_kernels.scipy_dist import ScipyDist

# step 1-- prepare a dataset (multivariate for demonstrating purposes)
X_train, y_train = load_arrow_head(split="train", return_X_y=True)
X_new, _ = load_arrow_head(split="test", return_X_y=True)

# step 2-- define a classification estimator with distance parameter IndepDist
indep_dist_sum = IndepDist(dist=FlatDist(ScipyDist()), aggfun="sum")
knn_sum = KNeighborsTimeSeriesClassifier(distance=indep_dist_sum)

# step 3-- train the models with the data
knn_sum.fit(X_train, y_train)

# step 4-- predict labels on new data
y_pred = knn_sum.predict(X_new[:5])  # Smaller set for faster runtime.

`IndepDist` provides the choice of being able to select `aggfun` parameter out of the many choices - `sum`, `mean`, `median`, `max`, `min`.

In [ ]:
print("Prediction with sum aggregation: ", y_pred)

Below is another implementation using `CombinedDistance` which applies `DtwDist()` on separate components.

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head
from sktime.dists_kernels.algebra import CombinedDistance
from sktime.dists_kernels.compose_tab_to_panel import AggrDist, FlatDist
from sktime.dists_kernels.scipy_dist import ScipyDist

# step 1-- load a dataset
X_train, y_train = load_arrow_head(split="train", return_X_y=True)
X_new, _ = load_arrow_head(split="test", return_X_y=True)

# step 2-- setup combined distance
combined_dist = CombinedDistance([FlatDist(ScipyDist()), AggrDist(ScipyDist())])

## step 3-- initialize the classifier
knn_combined_dist = KNeighborsTimeSeriesClassifier(distance=combined_dist)

## step 4-- train the classifier
knn_combined_dist.fit(X_train, y_train)

## step 5-- obtain the predictions from the classifier.
y_pred = knn_combined_dist.predict(X_new[:5])

In [ ]:
y_pred

### 2.2.6 Time Series Regression - basic vignettes

TSR vignettes are exactly the same as TSC, except that:

* `y` in `fit` input and `predict` output should be float 1D `np.ndarray`, not categorical
* other algorithms are commonly used and/or performant

In [ ]:
# steps 1, 2 - prepare dataset (train and new)
from sktime.datasets import load_covid_3month

X_train, y_train = load_covid_3month(split="train")
y_train = y_train.astype("float")
X_new, _ = load_covid_3month(split="test")
X_new = X_new.loc[:2]  # smaller dataset for faster notebook runtime

# step 3 - specify the regressor
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

clf = KNeighborsTimeSeriesRegressor(n_neighbors=3, distance=eucl_dist)

# step 4 - fit/train the regressor
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [ ]:
y_pred  # predictions are array of float

### 2.2.7 Time Series Clustering - basic vignettes

TS clustering is similar - 1st step is also `fit`, but unsupervised

i.e., no labels `y`, and next step is inspecting clusters

In [ ]:
# step 1 - prepare dataset (train and new)
from sktime.datasets import load_italy_power_demand

X, _ = load_italy_power_demand(split="train", return_type="numpy3D")

# step 2 - specify the clusterer
from sktime.clustering.dbscan import TimeSeriesDBSCAN
from sktime.dists_kernels import FlatDist, ScipyDist

eucl_dist = FlatDist(ScipyDist())
clst = TimeSeriesDBSCAN(distance=eucl_dist, eps=2)

# step 3 - fit the clusterer to the data
clst.fit(X)

# step 4 - inspect the clustering
clst.get_fitted_params()

## 2.3 Searching for estimators, estimator tags

Estimators in `sktime` are tagged.

Tags starting with "capability" indicate things the estimator can or cannot do, e.g.,

* `"capability:missing_values"` - dealing with missing values
* `"capability:multivariate"` - dealing with multivariate input
* `"capability:unequal_length"` - deaing with time series panels where the individual time series have unequal length and/or unequal index

all tags for an estimator scitype (e.g., classifier, regressor) can be inspected by `sktime.registry.all_tags`:

In [ ]:
from sktime.registry import all_tags

all_tags("classifier", as_dataframe=True)

valid estimator types are listed in the `all_tags` docstring, or `sktime.registry.BASE_CLASS_REGISTER`

In [ ]:
from sktime.registry import get_obj_scitype_list

# get only fist table column, the list of types
get_obj_scitype_list()

to find all estimators of a certain type, use `sktime.registry.all_estimators`

In [ ]:
# list all classifiers in sktime
from sktime.registry import all_estimators

all_estimators("classifier", as_dataframe=True)

for listing all estimators of a certain type with a certain capability,
use the `filter_tags` argument of `all_estimators`:

In [ ]:
# list all classifiers in sktime
# that can classify panels of time series containing missing data
from sktime.registry import all_estimators

all_estimators(
    "classifier",
    as_dataframe=True,
    filter_tags={"capability:missing_values": True},
)

side note:

don't worry about how short the list is - when in doubt, it is always possible to pipeline with `Imputer`

as in the next section :-)

## 2.4 Pipelines, Feature Extraction, Tuning, Composition


similar to `sklearn` for "tabular" classification, regression, etc,

`sktime` has a rich set of tools for:

* feature extraction via transformers
* pipeline transformers with any estimator
* tuning individual estimators or pipelines via grid search and similar
* building ensembles out of individual estimators, or other composites

`sktime` is also fully interoperable with `sklearn` interface if `numpy` based data mtypes are used

(although this loses support for unequal length time series)

### 2.4.1 Primer on `sktime` transformers for feature extraction

all `sktime` transformers work natively with panel data:

In [ ]:
from sktime.datasets import load_italy_power_demand
from sktime.transformations.series.detrend import Detrender

# load some panel data
X, _ = load_italy_power_demand(return_type="pd-multiindex")

# specify a linear detrender
detrender = Detrender()

# detrend X by removing linear trend from each instance
X_detrended = detrender.fit_transform(X)
X_detrended

for panel tasks such as TSC, TSR, clustering, there are two distinctions to be aware of:

* series-to-series transformers transform individual series to series, panels to panels. E.g., instance-wise detrender above
* series-to-primitive transformers transform individual series to a set of tabular features. E>g., summary feature extractor

either type of transform can be instance-wise:

* instance-wise transforms use only the i-th series to transform the i-th series. E.g., instance-wise detrender
* non-instance-wise transforms train on all series to transform the i-th series. E.g., PCA, overall mean detrender

In [ ]:
# example of a series-to-primitive transformer
from sktime.transformations.series.summarize import SummaryTransformer

# specify summary transformer
summary_trafo = SummaryTransformer()

# extract summary features - one per instance in the panel
X_summaries = summary_trafo.fit_transform(X)
X_summaries

just like classifiers, we can search for transformers of either type via the right tag:

* `"scitype:transform-input"` and `"scitype:transform-output"` define input and output, e.g., "series-to-series" (both are scitype strings)
* `"scitype:instancewise"` is boolean and tells us whether the transform is instance-wise

In [ ]:
# example: looking for all series-to-primitive transformers that are instance-wise
from sktime.registry import all_estimators

all_estimators(
    "transformer",
    as_dataframe=True,
    filter_tags={
        "scitype:transform-input": "Series",
        "scitype:transform-output": "Primitives",
        "scitype:instancewise": True,
    },
)

Further details on transformations and feature extraction can be found in the tutorial 3, transformers.

All composition steps therein (e.g., chaining, column subsetting) work together with all estimator types in `sktime`, including classifiers, regressors, clusterers.

### 2.4.2 Pipelines for time series panel tasks

all panel estimators pipeline with `sktime` transformers, via the `*` dunder or `make_pipeline`.

The pipeline does the following:

* in `fit`: runs the transformers' `fit_transform` in sequence, then `fit` of the panel estimator
* in `predict`, runs the fitted transformers' `transform` in sequence, then `predict` of the panel estimator

(the logic is same as for `sklearn` pipelines)

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.transformations.series.exponent import ExponentTransformer

pipe = ExponentTransformer() * KNeighborsTimeSeriesClassifier()

# this constructs a ClassifierPipeline, which is also a classifier
pipe

In [ ]:
# alternative to construct:
from sktime.pipeline import make_pipeline

pipe = make_pipeline(ExponentTransformer(), KNeighborsTimeSeriesClassifier())

In [ ]:
from sktime.datasets import load_unit_test

X_train, y_train = load_unit_test(split="TRAIN")
X_test, _ = load_unit_test(split="TEST")

# this is a ClassifierPipeline with the same interface as knn-classifier
# first applies exponent transform, then knn-classifier
pipe.fit(X_train, y_train)

`sktime` transformers pipeline with `sklearn` classifiers!

This allows to build "time series feature extraction then `sklearn` classify`" pipelines:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sktime.transformations.series.summarize import SummaryTransformer

# specify summary transformer
summary_rf = SummaryTransformer() * RandomForestClassifier()

summary_rf.fit(X_train, y_train)

### 2.4.3 Using transformers to deal with unequal length or missing values

pro tip: useful transformers to pipeline are those that "improve" capabilities!

Search for these transformer tags:

* `"capability:unequal_length:removes"` - ensures all instances in the panel have equal length afterwards. Examples: padding, cutting, resampling.
* `"capability:missing_values:removes"` - removes all missing values from the data (e.g., series, panel) passed to it. Example: mean imputation

In [ ]:
# all transformers that guarantee that the output is equal length and equal index
from sktime.registry import all_estimators

all_estimators(
    "transformer",
    as_dataframe=True,
    filter_tags={"capability:unequal_length:removes": True},
)

In [ ]:
# all transformers that guarantee the output has no missing values
from sktime.registry import all_estimators

all_estimators(
    "transformer",
    as_dataframe=True,
    filter_tags={"capability:missing_values:removes": True},
)

minor note:

some transformers guarantee "no missing values" under some conditions but not always, e.g., `TimeBinAggregate`

let's check the tags in one example

In [ ]:
# list all classifiers in sktime
from sktime.classification.feature_based import SummaryClassifier

no_missing_clf = SummaryClassifier()

no_missing_clf.get_tags()

In [ ]:
from sktime.transformations.series.impute import Imputer

clf_can_do_missing = Imputer() * SummaryClassifier()

clf_can_do_missing.get_tags()

### 2.4.4 Tuning and model selection

`sktime` classifiers are compatible with `sklearn` model selection and composition tools using `sktime` data formats.

This extends to grid tuning and cross-validation, as long as `numpy` based formats or length/instance indexed formats are used.

In [ ]:
from sktime.datasets import load_unit_test

X_train, y_train = load_unit_test(split="TRAIN")
X_test, _ = load_unit_test(split="TEST")

Cross-validation using the `sklearn` `cross_val_score` and `KFold` functionality:

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

from sktime.classification.feature_based import SummaryClassifier

clf = SummaryClassifier()

cross_val_score(clf, X_train, y=y_train, cv=KFold(n_splits=4))

Parameter tuning using `sklearn` `GridSearchCV`, we tune the _k_ and distance measure for a K-NN classifier:

In [ ]:
from sklearn.model_selection import GridSearchCV

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

knn = KNeighborsTimeSeriesClassifier()
param_grid = {"n_neighbors": [1, 5], "distance": ["euclidean", "dtw"]}
parameter_tuning_method = GridSearchCV(knn, param_grid, cv=KFold(n_splits=4))

parameter_tuning_method.fit(X_train, y_train)
y_pred = parameter_tuning_method.predict(X_test)

### 2.4.5 Advanced Composition cheat sheet - AutoML, bagging, ensembles

* common ensembling patterns: `BaggingClassifier`, `WeightedEnsembleClassifier`
* composability with `sklearn` classifier, regressor building blocks still applies
* AutoML can be achieved by combining tuning with `MultiplexClassifier` or `MultiplexTransformer`

pro tip: bagging with a fixed single column subset can be used to turn an univariate classifier into a multivariate classifier!

## 2.5 Appendix - Extension guide

`sktime` is meant to be easily extensible, for direct contribution to `sktime` as well as for local/private extension with custom methods.

To extend `sktime` with a new local or contributed estimator, a good workflow to follow is:

0. find the right extension template for the type of estimator you want to add - e.g., classifier, regressor, clusterer, etc. The extension templates are located in the [`extension_templates](https://github.com/sktime/sktime/blob/main/extension_templates) directory
1. read through the extension template - this is a `python` file with `todo` blocks that mark the places in which changes need to be added.
2. optionally, if you are planning any major surgeries to the interface: look at the base class - note that "ordinary" extension (e.g., new algorithm) should be easily doable without this.
3. copy the extension template to a local folder in your own repository (local/private extension), or to a suitable location in your clone of the `sktime` or affiliated repository (if contributed extension), inside `sktime.[name_of_task]`; rename the file and update the file docstring appropriately.
4. address the "todo" parts. Usually, this means: changing the name of the class, setting the tag values, specifying hyper-parameters, filling in `__init__`, `_fit`, `_predict` and/or other methods (for details see the extension template). You can add private methods as long as they do not override the default public interface. For more details, see the extension template.
5. to test your estimator manually: import your estimator and run it in the basic vignettes above.
6. to test your estimator automatically: call `sktime.tests.test_all_estimators.check_estimator` on your estimator. You can call this on a class or object instance. Ensure you have specified test parameters in the `get_test_params` method, according to the extension template.

In case of direct contribution to `sktime` or one of its affiliated packages, additionally:
* Add yourself as an author and/or a maintainer for the new estimator file(s), via `"authors"` and `"maintainers"` tag.
* create a pull request that contains only the new estimators (and their inheritance tree, if it's not just one class), as well as the automated tests as described above.
* in the pull request, describe the estimator and optimally provide a publication or other technical reference for the strategy it implements.
* before making the pull request, ensure that you have all necessary permissions to contribute the code to a permissive license (BSD-3) open source project.